<a href="https://colab.research.google.com/github/mahinthjoe/Code/blob/master/backhat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Travesura blackhat 🎩:

1. Utiliza uno o varios headings (ej. misHeadings = ["fiebre", "causas de la fiebre"])
2. Si sólo incluyes un heading el script completará los encabezados con las preguntas frecuentes de google.
3. Define idioma original e idioma traducido
4. Al pulsar el play el script buscará cada headings en el idioma traducido y recuperará el fragmento de texto más similar.

<br>

↓ Los resultados aparecerán debajo del código ↓ 

<br>

Un saludo desde Mallorca,

[Jose Gris](https://twitter.com/JoseGrisSEO) 😎

<br>

---

Si te son útiles mis colabs [invítame a un café](https://www.buymeacoffee.com/josegris)

---

In [ ]:
#HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
# Parámetros principales
# Si sólo incluyes un heading recuperará PAA (preguntas frecuentes) de google y las usará también como headings
misHeadings = ["¿Qué puede comer un perro?"] # ej. misHeadings = ["fiebre", "causas de la fiebre", "Sintomas de la fiebre"]
idiomaOriginal = 'es' # => Acepta cualquier idioma 'es' (español), 'en' (inglés)...
paisOriginal = 'es' # España => por ejemplo 'us' (USA), 'uk' (Inglaterra), 'fr' Francia...
idiomaTraducido = "it" # => acepta 'es' (español), 'en' (inglés), 'fr' (francés), 'de' (alemán), 'it' (italiano)
paisTraducido = 'it' # España => por ejemplo 'us' (USA), 'uk' (Inglaterra), 'fr' Francia, 'it' (Italia), 'de' (Alemania)...
#HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
# Parámetros opcionales
numArticulos = 1 # número de artículos que mostrará para cada heading
numMaximoHeadingsPorArticulo = 2 # Número de headings que mostrará por cada artículo
numParrafosMaximo = 3 # Párrafos que recupera para cada heading
minimaPuntuacion = 0.4 # Mínima similitud entre keyword y heading
#HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH


urlsUsadas = []
from bs4 import BeautifulSoup
import requests

def scrapearSERP(keyword, idioma, pais):
  # aquí podría devolver también el texto del featured snippet
  urls = []
  URL = f"https://www.google.com/search?hl={idioma}&gl={pais}&q={keyword}&oq={keyword}"
  headers =  {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"}
  resp = requests.get(URL, headers = headers)
  if resp.status_code == 200: 
    soup = BeautifulSoup(resp.content, "html.parser")
    links = soup.find_all("div", {"class" : "g"})
    for x in links:
      links0 = x.find_all("a", href=True)
      if len(links0) == 0:continue
      link = links0[0]['href']
      #corrijo feature snippet
      if "#:~:text" in link: 
        link = link.split("#:~:text")[0]
      if link.startswith("/"):continue
      if link not in urls: urls.append(link)

      preguntas = soup.find_all("div", {"jsname" : "jIA8B"})
      preguntas = [x.getText() for x in preguntas]
  
    return urls, preguntas
  else:
    print(f"Google devuelve un código de respuesta {resp.status_code}")
    return []


!pip install requests
!pip install justext
import justext

! pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch
from textblob import TextBlob

modelo = ''
stopeee = ''

if idiomaTraducido == 'en':
  modelo = 'sentence-transformers/all-MiniLM-L6-v2'
  stopeee = 'English'
elif idiomaTraducido == 'es':
  modelo = 'hiiamsid/sentence_similarity_spanish_es'
  stopeee = 'Spanish'
elif idiomaTraducido == 'fr':
  modelo = "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"
  stopeee = 'French'
elif idiomaTraducido == 'de':
  modelo = 'Sahajtomar/German-semantic'
  stopeee = 'German'
elif idiomaTraducido == 'it':
  modelo = 'efederici/sentence-bert-base'
  stopeee = 'Italian'


embedder = SentenceTransformer(modelo) 

def traducir(texto, idioma0, idioma1):
  if idioma0 == idioma1: return texto
  analysis = TextBlob(texto)
  textoTraducido = analysis.translate(from_lang=idioma0, to=idioma1).string
  return(textoTraducido.replace(' ...', '...'))

def textosSemanticamenteParecidos(query, url):

  try:
    response = requests.get(url)
    if response.status_code < 200 or response.status_code >= 400:
      # print(f"sorry codigo de respuesta {response.status_code}")
      return []
    paragraphs = justext.justext(response.content, justext.get_stoplist(stopeee)) #English
    
    noValidos = [p.text for p in paragraphs if p.is_boilerplate] #and len(p.text) < 70
    textos = [p for p in paragraphs]
    textos = [[paragraph.xpath.split("/")[-1].split("[")[0], paragraph.text] for paragraph in textos]
    textos = [p for p in textos if (p[0].upper().startswith("H") or p[1] not in noValidos) and p[0].upper() != "DIV"]

    
    # junto los elementos de lista con su punto anterior
    hola = textos.copy()
    hola = [x for x in hola if x[0].upper().startswith("H")]
    
    if len(hola) == 0: 
      # print("No se han encontrado headings")
      return []
    final = []
    corpus_embeddings = embedder.encode(hola, convert_to_tensor=True)
    queries = [query]


    # # Buscamos los headings más similares
    top_k = min(numMaximoHeadingsPorArticulo+3, len(hola))
    for query in queries:
        resultado = []
        query_embedding = embedder.encode(query, convert_to_tensor=True)

        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
        hits = hits[0]
        for hit in hits:
            resultado.append([hola[hit['corpus_id']], hit['score']])
           
        final.append(resultado)
   
    aDevolver = final[0]
    
    
    for x in aDevolver:
      if x[1]< minimaPuntuacion:continue
      extra = []
      indice = textos.index(x[0])
      if indice + 1 == len(textos):
        continue  
      
      contador = 0

      for y in textos[indice+1:]:
        if y[0] != 'li': contador = contador +1
        if y[0].upper().startswith("H"): break
        extra.append(y)
        
        if contador >= numParrafosMaximo:break
        
      x.append(extra)
    
    aDevolver = [x for x in aDevolver if len(x) > 2 and len(x[2]) > 0]
    aDevolver = aDevolver[:numMaximoHeadingsPorArticulo]
    
    if len(aDevolver) == 0:
      # print("Sorry no se ha encontrado ningún heading similar con texto") #aquí podría probar directamente con las frases
      return []
    
    urlsUsadas.append(url)

    print(" ")
    print(url)

    for r in aDevolver:
      print("------   " + str(round(r[1], 2)) + "   ------")
      print(f"<{r[0][0]}>" + traducir(r[0][1], idiomaTraducido, idiomaOriginal) +f"</{r[0][0]}>" + " - ")
      for p in r[2]:
        print(f"<{p[0]}>" + traducir(p[1], idiomaTraducido, idiomaOriginal) +f"</{p[0]}>" )
        


    return aDevolver
      
  except:
    return []
  
print("Headings:", misHeadings)
print("Idioma Original:", idiomaOriginal)
print("País Original:", paisOriginal)
print("----------------------------------")
print("Idioma Traducido:", idiomaTraducido)
print("País Traducido:", paisTraducido)
print("----------------------------------")

if len(misHeadings) == 1:
  urls, preguntas = scrapearSERP(misHeadings[0], idiomaOriginal, paisOriginal)
  misHeadings.extend(preguntas)
  misHeadings
  print("PAA como headings:")
  print(preguntas)


for keyword0 in misHeadings:
  print(" ")
  print("==============================================")
  print(keyword0)
  print("==============================================")

  keywordTraducida = traducir(keyword0, idiomaOriginal, idiomaTraducido)

  urls0, preguntas = scrapearSERP(keywordTraducida, idiomaTraducido, paisTraducido)

  # busco similitud semántica de headings y su texto para cada url
  contador = 0
  for url in urls0:
    if url in urlsUsadas:continue # paso de usar dos veces la misma url
    try:
      resultado0 = textosSemanticamenteParecidos(keywordTraducida, url)
      if len(resultado0) == 0:continue
      contador = contador + 1
      if contador == numArticulos:break # uso un máximo número de artículos
    except:
      True
    #  print(f"error parseando {url}")
    


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/635 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/725k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/373 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Headings: ['¿Qué puede comer un perro?']
Idioma Original: es
País Original: es
----------------------------------
Idioma Traducido: it
País Traducido: it
----------------------------------
PAA como headings:
['¿Qué comida de humanos se le puede dar a los perros?', '¿Qué es lo que no pueden comer los perros?', '¿Qué fruta se le puede dar a un perro?', '¿Qué comen los perros naturalmente?']
 
¿Qué puede comer un perro?
 
https://www.petyoo.it/blog/cani/cosa-possono-mangiare-i-cani.html
------   0.58   ------
<h2>🥬 ¿Pueden los perros comer lechuga? ¿Puedo dar espinacas a mi perro? ¡Por supuesto!</h2> - 
<p>La mayoría de las verduras verdes con hojas son buenas para los perros tanto como para los hombres. De hecho, muchos de ellos entran en los ingredientes de la comida para perros. Los más comunes son el repollo rizado y acelgado, pero las espinacas también se usan de vez en cuando.</p>
<p>Estas plantas muy útiles contienen una pila de vitamina A, K y C.</p>
<p>Las espinacas contienen un 